# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 18 2022 8:56AM,237496,15,8031050,"Laser Pharmaceuticals, LLC",Released
1,May 18 2022 8:56AM,237496,15,8031053,"Laser Pharmaceuticals, LLC",Released
2,May 18 2022 8:56AM,237496,15,8031683,"Laser Pharmaceuticals, LLC",Released
3,May 18 2022 8:56AM,237496,15,8031684,"Laser Pharmaceuticals, LLC",Released
4,May 18 2022 8:56AM,237496,15,8031685,"Laser Pharmaceuticals, LLC",Released
5,May 18 2022 8:56AM,237496,15,8031695,"Laser Pharmaceuticals, LLC",Released
6,May 18 2022 8:56AM,237496,15,8031696,"Laser Pharmaceuticals, LLC",Released
7,May 18 2022 8:56AM,237496,15,8031697,"Laser Pharmaceuticals, LLC",Released
8,May 18 2022 8:56AM,237496,15,8031698,"Laser Pharmaceuticals, LLC",Released
9,May 18 2022 8:56AM,237496,15,8031699,"Laser Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
68,237500,Released,18
69,237501,Released,14
70,237502,Released,2
71,237503,Released,7
72,237504,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237500,NaN,NaN,18.0
237501,NaN,NaN,14.0
237502,NaN,NaN,2.0
237503,NaN,NaN,7.0
237504,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0.0,1.0,0.0
237385,0.0,0.0,1.0
237386,0.0,0.0,1.0
237388,0.0,0.0,1.0
237394,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0,1,0
237385,0,0,1
237386,0,0,1
237388,0,0,1
237394,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,0,1,0
1,237385,0,0,1
2,237386,0,0,1
3,237388,0,0,1
4,237394,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,,1,
1,237385,,,1
2,237386,,,1
3,237388,,,1
4,237394,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 18 2022 8:56AM,237496,15,"Laser Pharmaceuticals, LLC"
18,May 18 2022 8:55AM,237497,15,"Carwin Pharmaceutical Associates, LLC"
27,May 18 2022 8:55AM,237495,15,"Brookfield Pharmaceuticals, LLC"
29,May 18 2022 8:55AM,237504,10,"Citieffe, Inc."
30,May 18 2022 8:53AM,237503,10,"Amcyte Pharma, Inc."
37,May 18 2022 8:50AM,237502,10,Emerginnova
39,May 18 2022 8:50AM,237501,10,ISDIN Corporation
53,May 18 2022 8:47AM,237500,20,Alumier Labs Inc.
71,May 18 2022 8:38AM,237499,10,"Citieffe, Inc."
72,May 18 2022 8:24AM,237494,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 18 2022 8:56AM,237496,15,"Laser Pharmaceuticals, LLC",,,18
1,May 18 2022 8:55AM,237497,15,"Carwin Pharmaceutical Associates, LLC",,,9
2,May 18 2022 8:55AM,237495,15,"Brookfield Pharmaceuticals, LLC",,,2
3,May 18 2022 8:55AM,237504,10,"Citieffe, Inc.",,,1
4,May 18 2022 8:53AM,237503,10,"Amcyte Pharma, Inc.",,,7
5,May 18 2022 8:50AM,237502,10,Emerginnova,,,2
6,May 18 2022 8:50AM,237501,10,ISDIN Corporation,,,14
7,May 18 2022 8:47AM,237500,20,Alumier Labs Inc.,,,18
8,May 18 2022 8:38AM,237499,10,"Citieffe, Inc.",,,1
9,May 18 2022 8:24AM,237494,15,"Alliance Pharma, Inc.",,,20


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 8:56AM,237496,15,"Laser Pharmaceuticals, LLC",18,,
1,May 18 2022 8:55AM,237497,15,"Carwin Pharmaceutical Associates, LLC",9,,
2,May 18 2022 8:55AM,237495,15,"Brookfield Pharmaceuticals, LLC",2,,
3,May 18 2022 8:55AM,237504,10,"Citieffe, Inc.",1,,
4,May 18 2022 8:53AM,237503,10,"Amcyte Pharma, Inc.",7,,
5,May 18 2022 8:50AM,237502,10,Emerginnova,2,,
6,May 18 2022 8:50AM,237501,10,ISDIN Corporation,14,,
7,May 18 2022 8:47AM,237500,20,Alumier Labs Inc.,18,,
8,May 18 2022 8:38AM,237499,10,"Citieffe, Inc.",1,,
9,May 18 2022 8:24AM,237494,15,"Alliance Pharma, Inc.",20,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 8:56AM,237496,15,"Laser Pharmaceuticals, LLC",18,,
1,May 18 2022 8:55AM,237497,15,"Carwin Pharmaceutical Associates, LLC",9,,
2,May 18 2022 8:55AM,237495,15,"Brookfield Pharmaceuticals, LLC",2,,
3,May 18 2022 8:55AM,237504,10,"Citieffe, Inc.",1,,
4,May 18 2022 8:53AM,237503,10,"Amcyte Pharma, Inc.",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 8:56AM,237496,15,"Laser Pharmaceuticals, LLC",18.0,NaN,NaN
1,May 18 2022 8:55AM,237497,15,"Carwin Pharmaceutical Associates, LLC",9.0,NaN,NaN
2,May 18 2022 8:55AM,237495,15,"Brookfield Pharmaceuticals, LLC",2.0,NaN,NaN
3,May 18 2022 8:55AM,237504,10,"Citieffe, Inc.",1.0,NaN,NaN
4,May 18 2022 8:53AM,237503,10,"Amcyte Pharma, Inc.",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 8:56AM,237496,15,"Laser Pharmaceuticals, LLC",18.0,0.0,0.0
1,May 18 2022 8:55AM,237497,15,"Carwin Pharmaceutical Associates, LLC",9.0,0.0,0.0
2,May 18 2022 8:55AM,237495,15,"Brookfield Pharmaceuticals, LLC",2.0,0.0,0.0
3,May 18 2022 8:55AM,237504,10,"Citieffe, Inc.",1.0,0.0,0.0
4,May 18 2022 8:53AM,237503,10,"Amcyte Pharma, Inc.",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3561877,100.0,20.0,34.0
15,1187423,49.0,1.0,0.0
16,949731,3.0,1.0,0.0
18,5461132,22.0,1.0,0.0
19,1424779,28.0,8.0,0.0
20,2374277,70.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3561877,100.0,20.0,34.0
1,15,1187423,49.0,1.0,0.0
2,16,949731,3.0,1.0,0.0
3,18,5461132,22.0,1.0,0.0
4,19,1424779,28.0,8.0,0.0
5,20,2374277,70.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,100.0,20.0,34.0
1,15,49.0,1.0,0.0
2,16,3.0,1.0,0.0
3,18,22.0,1.0,0.0
4,19,28.0,8.0,0.0
5,20,70.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,100.0
1,15,Released,49.0
2,16,Released,3.0
3,18,Released,22.0
4,19,Released,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Completed,34.0,0.0,0.0,0.0,0.0,0.0
Executing,20.0,1.0,1.0,1.0,8.0,1.0
Released,100.0,49.0,3.0,22.0,28.0,70.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Completed,34.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,1.0,1.0,1.0,8.0,1.0
2,Released,100.0,49.0,3.0,22.0,28.0,70.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Completed,34.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,1.0,1.0,1.0,8.0,1.0
2,Released,100.0,49.0,3.0,22.0,28.0,70.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()